### Build a Question Answering application over a Graph Database

In [1]:
NEO4J_URI="neo4j+s://b0ca95b1.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="rzk9c_6_7ClcpChs379evwH1o_BbpHyKp1AEeIjlQXM"

In [2]:
# # personal id 
# NEO4J_URI="neo4j+s://b2f94349.databases.neo4j.io"
# NEO4J_USERNAME="neo4j"
# NEO4J_PASSWORD="KtzFPvWb1XHHHvi3XbU0Jx9JCVg98Feacn9cHoM1b9g"

In [3]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [4]:
# from neo4j import GraphDatabase

# driver = GraphDatabase.driver("neo4j+s://b0ca95b1.databases.neo4j.io", auth=("neo4j", "rzk9c_6_7ClcpChs379evwH1o_BbpHyKp1AEeIjlQXM"))

# try:
#     with driver.session() as session:
#         result = session.run("RETURN 1")
#         print("Connected to Neo4j:", result.single())
# finally:
    # driver.close()


In [5]:
!pip install langchain_community


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

Unable to retrieve routing information


ValueError: Could not connect to Neo4j database. Please ensure that the url is correct

In [ ]:
# from langchain_community.graphs import Neo4jGraph
# graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
# graph

In [ ]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [ ]:
moview_query

In [ ]:
graph.query(moview_query)

In [ ]:
graph.refresh_schema()
print(graph.schema)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

In [ ]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

In [ ]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response

In [ ]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response

In [ ]:
response=chain.invoke({"query":"How many artists are there?"})
response

In [ ]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response